In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

# Check names.
student_data_df.head(10)

## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [ ]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [ ]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[(student_data_df['school_name']=='Thomas High School') & (student_data_df['grade'] == '9th'), 'reading_score'] = np.NaN



In [ ]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[(student_data_df['school_name']=='Thomas High School')& (student_data_df['grade']=='9th'),'math_score'] = np.NaN

In [ ]:
#  Step 4. Check the student data for NaN's. 

student_data_df.loc[(student_data_df['school_name']=='Thomas High School')& (student_data_df['grade']=='9th'),['school_name','grade','reading_score','math_score']]

## Deliverable 2 : Repeat the school district analysis

### District Summary

In [ ]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df

In [ ]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [ ]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [ ]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
thomas_9th_grader_count = student_data_df.loc[(student_data_df['school_name']=='Thomas High School')&(student_data_df['grade']=='9th'),'student_name'].count()

# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()


# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
new_student_count = student_count-thomas_9th_grader_count


In [ ]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

In [ ]:
# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage = float(passing_math_count)/float(new_student_count)*100
passing_reading_percentage = float(passing_reading_count)/float(new_student_count)*100

In [ ]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()


# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage = float(overall_passing_math_reading_count)/float(new_student_count)*100

In [ ]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": new_student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

##  School Summary

In [ ]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [ ]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})


# per_school_summary_df

In [ ]:
# # Format the Total School Budget and the Per Student Budget
# per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
# per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# # Display the data frame
per_school_summary_df

In [ ]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
thomas_student_count = student_data_df.loc[student_data_df['school_name']=='Thomas High School','student_name'].count() - student_data_df.loc[(student_data_df['school_name']=='Thomas High School')&(student_data_df['grade']=='9th'),'student_name'].count()

In [ ]:
# Step 6. Get all the students passing math from THS
thomas_passing_reading = school_data_complete_df[(school_data_complete_df["school_name"] =='Thomas High School') &(school_data_complete_df["math_score"] >= 70)]

In [ ]:
# Step 7. Get all the students passing reading from THS
thomas_passing_reading = school_data_complete_df[(school_data_complete_df["school_name"] =='Thomas High School') &(school_data_complete_df["reading_score"] >= 70)]

In [ ]:
# Step 8. Get all the students passing math and reading from THS
thomas_passing_reading = school_data_complete_df[(school_data_complete_df["school_name"] =='Thomas High School')&(school_data_complete_df["math_score"] >= 70) &(school_data_complete_df["reading_score"] >= 70)]

In [ ]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 
thomas_ten_twelfth_maths = school_data_complete_df[(school_data_complete_df["school_name"] =='Thomas High School')&(school_data_complete_df["grade"] != '9th') &(school_data_complete_df["math_score"] >= 70)].count()['student_name']
thomas_ten_twelfth_maths_percentage = float(thomas_ten_twelfth_maths)/float(thomas_student_count)*100

In [ ]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
thomas_ten_twelfth_reading = school_data_complete_df[(school_data_complete_df["school_name"] =='Thomas High School')&(school_data_complete_df["grade"] != '9th') &(school_data_complete_df["reading_score"] >= 70)].count()['student_name']
thomas_ten_twelfth_reading_percentage = float(thomas_ten_twelfth_reading)/float(thomas_student_count)*100

In [ ]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
thomas_ten_twelfth_overall = school_data_complete_df[(school_data_complete_df["school_name"] =='Thomas High School')&(school_data_complete_df["grade"] != '9th') &(school_data_complete_df["reading_score"] >= 70)&(school_data_complete_df["math_score"] >= 70)].count()['student_name']
thomas_ten_twelfth_overall_percentage = float(thomas_ten_twelfth_overall )/float(thomas_student_count)*100


In [ ]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc['Thomas High School','% Passing Math'] = thomas_ten_twelfth_maths_percentage

In [ ]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc['Thomas High School','% Passing Reading'] = thomas_ten_twelfth_reading_percentage

In [ ]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc['Thomas High School','% Overall Passing'] = thomas_ten_twelfth_overall_percentage

In [ ]:
# per_school_summary_df['Total School Budget'] = per_school_summary_df['Total School Budget'].map('${:,.2f}'.format)
# per_school_summary_df['Per Student Budget'] = per_school_summary_df['Per Student Budget'].map('${:,.2f}'.format)
# per_school_summary_df['Total Students'] = per_school_summary_df['Total Students'].map('{:,}'.format)
# per_school_summary_df['Average Math Score'] = per_school_summary_df['Average Math Score'].map('{:.1f}'.format)
# per_school_summary_df['Average Reading Score'] = per_school_summary_df['Average Reading Score'].map('{:.1f}'.format)
# per_school_summary_df['% Passing Math'] = per_school_summary_df['% Passing Math'].map('{:.1f}'.format)
# per_school_summary_df['% Passing Reading'] = per_school_summary_df['% Passing Reading'].map('{:.1f}'.format)
# per_school_summary_df['% Overall Passing'] = per_school_summary_df['% Overall Passing'].map('{:.1f}'.format)
per_school_summary_df

## High and Low Performing Schools 

In [ ]:
# Sort and show top five schools
per_school_summary_df.sort_values('% Overall Passing',ascending =False).head(5)

In [ ]:
# Sort and show bottom five schools.
per_school_summary_df.sort_values('% Overall Passing',ascending =True).head(5)

## Math and Reading Scores by Grade

In [ ]:
# Create a Series of scores by grade levels using conditionals.
ninth_grade = school_data_complete_df[(school_data_complete_df['grade']=='9th')]
tenth_grade = school_data_complete_df[(school_data_complete_df['grade']=='10th')]
eleventh_grade = school_data_complete_df[(school_data_complete_df['grade']=='11th')]
twelfth_grade = school_data_complete_df[(school_data_complete_df['grade']=='12th')]

# Group each school Series by the school name for the average math score.
per_school_ninth_math = ninth_grade.groupby('school_name').mean()['math_score']
per_school_tenth_math = tenth_grade.groupby('school_name').mean()['math_score']
per_school_eleventh_math = eleventh_grade.groupby('school_name').mean()['math_score']
per_school_twelfth_math = twelfth_grade.groupby('school_name').mean()['math_score']

# Group each school Series by the school name for the average reading score.
per_school_ninth_reading = ninth_grade.groupby('school_name').mean()['reading_score']
per_school_tenth_reading = tenth_grade.groupby('school_name').mean()['reading_score']
per_school_eleventh_reading = eleventh_grade.groupby('school_name').mean()['reading_score']
per_school_twelfth_reading = twelfth_grade.groupby('school_name').mean()['reading_score']


In [ ]:
# Combine each Series for average math scores by school into single data frame.
average_math_scores_school = pd.DataFrame({'9th': per_school_ninth_math,
                                          '10th': per_school_tenth_math,
                                         '11th': per_school_eleventh_math,
                                         '12th' : per_school_twelfth_math})

In [ ]:
# Combine each Series for average reading scores by school into single data frame.
average_reading_scores_school = pd.DataFrame({'9th': per_school_ninth_reading,
                                          '10th': per_school_tenth_reading,
                                         '11th': per_school_eleventh_reading,
                                         '12th' : per_school_twelfth_reading})

In [ ]:
# Format each grade column.
average_math_scores_school['9th'] = average_math_scores_school['9th'].map("{:,.1f}".format)
average_math_scores_school['10th'] = average_math_scores_school['10th'].map("{:,.1f}".format)
average_math_scores_school['11th'] = average_math_scores_school['11th'].map("{:,.1f}".format)
average_math_scores_school['12th'] = average_math_scores_school['12th'].map("{:,.1f}".format)


In [ ]:
# Remove the index.
average_math_scores_school.index.name = None

# Display the data frame
average_math_scores_school

In [ ]:
# Format each grade column.
average_reading_scores_school['9th'] = average_reading_scores_school['9th'].map("{:,.1f}".format)
average_reading_scores_school['10th'] = average_reading_scores_school['10th'].map("{:,.1f}".format)
average_reading_scores_school['11th'] = average_reading_scores_school['11th'].map("{:,.1f}".format)
average_reading_scores_school['12th'] = average_reading_scores_school['12th'].map("{:,.1f}".format)


## Remove the index.
average_reading_scores_school.index.name= None

# Display the data frame
average_reading_scores_school

## Scores by School Spending

In [ ]:
# Establish the spending bins and group names.
spending_bins = [0,585,630,645,675]
spending_labels = ['<$584','$585-629','$630-644','$645-675']

# Categorize spending based on the bins.
per_school_summary_df['Spending Ranges (Per Student)'] = pd.cut(per_school_capita,spending_bins,labels =spending_labels)

In [ ]:
# Calculate averages for the desired columns. 
spending_math_scores = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['Average Math Score']
spending_reading_scores = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['Average Reading Score']
spending_passing_math = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Math']
spending_passing_reading = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Reading']
overall_passing_spending = per_school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Overall Passing']

In [ ]:
# Create the DataFrame
spending_summary_df = pd.DataFrame({'Average Math Score': spending_math_scores,
                                   'Average Reading Score': spending_reading_scores,
                                   '% Passing Maths': spending_passing_math,
                                   '% Passing Reading': spending_passing_reading,
                                   '% Overall Passing': overall_passing_spending})

In [ ]:
# Format the DataFrame 
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Maths"] = spending_summary_df["% Passing Maths"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map('{:,.1f}'.format)
spending_summary_df.index.name =None        
spending_summary_df

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0,1800,3000,5000]
size_labels = ['Small (<1800)','Medium (1800-3000)','Large (3000-5000)']

# Categorize spending based on the bins.
per_school_summary_df['School_Size'] = pd.cut(per_school_counts,size_bins, labels = size_labels) 

In [ ]:
# Calculate averages for the desired columns. 
size_math_score = per_school_summary_df.groupby(per_school_summary_df['School_Size']).mean()['Average Math Score']
size_reading_score = per_school_summary_df.groupby(per_school_summary_df['School_Size']).mean()['Average Reading Score']
size_passing_math = per_school_summary_df.groupby(per_school_summary_df['School_Size']).mean()['% Passing Math']
size_passing_reading= per_school_summary_df.groupby(per_school_summary_df['School_Size']).mean()['% Passing Reading']
size_passing_overall = per_school_summary_df.groupby(per_school_summary_df['School_Size']).mean()['% Overall Passing']

In [ ]:
# Assemble into DataFrame. 
school_size_summary = pd.DataFrame({'Average Math Score':size_math_score,
                                    "Average Reading Score": size_reading_score,
                                    "% Passing Math": size_passing_math, 
                                    "% Passing Reading": size_passing_reading,
                                    "% Overall Passing": size_passing_overall})
school_size_summary.index.name = None

In [ ]:
# Format the DataFrame  
school_size_summary['Average Math Score'] = school_size_summary['Average Math Score'].map('{:.1f}'.format)
school_size_summary['Average Reading Score'] = school_size_summary['Average Reading Score'].map('{:.1f}'.format)
school_size_summary['% Passing Math'] = school_size_summary['% Passing Math'].map('{:.0f}'.format)
school_size_summary['% Passing Reading'] = school_size_summary['% Passing Reading'].map('{:.0f}'.format)
school_size_summary['% Overall Passing'] = school_size_summary['% Overall Passing'].map('{:.0f}'.format)
school_size_summary

## Scores by School Type

In [ ]:
# Calculate averages for the desired columns.
type_math_score = per_school_summary_df.groupby(per_school_summary_df['School Type']).mean()['Average Math Score']
type_reading_score = per_school_summary_df.groupby(per_school_summary_df['School Type']).mean()['Average Reading Score']
type_passing_math = per_school_summary_df.groupby(per_school_summary_df['School Type']).mean()['% Passing Math']
type_passing_reading= per_school_summary_df.groupby(per_school_summary_df['School Type']).mean()['% Passing Reading']
type_passing_overall = per_school_summary_df.groupby(per_school_summary_df['School Type']).mean()['% Overall Passing']

In [ ]:
# Assemble into DataFrame. 
school_type_summary = pd.DataFrame({'Average Math Score':type_math_score,
                                    "Average Reading Score": type_reading_score,
                                    "% Passing Math": type_passing_math, 
                                    "% Passing Reading": type_passing_reading,
                                    "% Overall Passing": type_passing_overall})
school_type_summary.index.name = None

In [ ]:
# # Format the DataFrame 
school_type_summary['Average Math Score'] = school_type_summary['Average Math Score'].map('{:.1f}'.format)
school_type_summary['Average Reading Score'] = school_type_summary['Average Reading Score'].map('{:.1f}'.format)
school_type_summary['% Passing Math'] = school_type_summary['% Passing Math'].map('{:.0f}'.format)
school_type_summary['% Passing Reading'] = school_type_summary['% Passing Reading'].map('{:.0f}'.format)
school_type_summary['% Overall Passing'] = school_type_summary['% Overall Passing'].map('{:.0f}'.format)
school_type_summary